In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Training for Walmart Demand Prediction\n",
    "\n",
    "This notebook demonstrates the training of demand prediction models using historical sales data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score\n",
    "import joblib\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load and Prepare Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Load processed data\n",
    "df = pd.read_csv('../data/processed_sales_data.csv')\n",
    "df['date'] = pd.to_datetime(df['date'])\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Feature Engineering"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Create additional features\n",
    "df['year'] = df['date'].dt.year\n",
    "df['month'] = df['date'].dt.month\n",
    "df['day_of_week'] = df['date'].dt.dayofweek\n",
    "df['quarter'] = df['date'].dt.quarter\n",
    "\n",
    "# Create lag features\n",
    "df['lag_7'] = df.groupby(['region', 'product_id'])['sales_quantity'].shift(7)\n",
    "df['lag_14'] = df.groupby(['region', 'product_id'])['sales_quantity'].shift(14)\n",
    "df['lag_30'] = df.groupby(['region', 'product_id'])['sales_quantity'].shift(30)\n",
    "\n",
    "# Create rolling mean features\n",
    "df['rolling_mean_7'] = df.groupby(['region', 'product_id'])['sales_quantity'].transform(lambda x: x.rolling(7, min_periods=1).mean())\n",
    "df['rolling_mean_14'] = df.groupby(['region', 'product_id'])['sales_quantity'].transform(lambda x: x.rolling(14, min_periods=1).mean())\n",
    "\n",
    "# Drop rows with NaN values (from lag features)\n",
    "df = df.dropna()\n",
    "\n",
    "# Prepare features and target\n",
    "feature_columns = [\n",
    "    'month', 'day_of_week', 'quarter', 'year',\n",
    "    'lag_7', 'lag_14', 'lag_30',\n",
    "    'rolling_mean_7', 'rolling_mean_14',\n",
    "    'price', 'promotion_flag'\n",
    "]\n",
    "\n",
    "X = pd.get_dummies(df[feature_columns], columns=['month', 'day_of_week', 'quarter'])\n",
    "y = df['sales_quantity']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Train-Test Split"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Split data by time\n",
    "train_size = int(len(df) * 0.8)\n",
    "X_train = X[:train_size]\n",
    "X_test = X[train_size:]\n",
    "y_train = y[:train_size]\n",
    "y_test = y[train_size:]\n",
    "\n",
    "print(f\"Training set size: {len(X_train)}\")\n",
    "print(f\"Test set size: {len(X_test)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Model Training"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Initialize and train model\n",
    "model = RandomForestRegressor(\n",
    "    n_estimators=100,\n",
    "    max_depth=10,\n",
    "    random_state=42\n",
    ")\n",
    "\n",
    "model.fit(X_train, y_train)\n",
    "\n",
    "# Make predictions\n",
    "y_pred = model.predict(X_test)\n",
    "\n",
    "# Calculate metrics\n",
    "mae = mean_absolute_error(y_test, y_pred)\n",
    "mse = mean_squared_error(y_test, y_pred)\n",
    "r2 = r2_score(y_test, y_pred)\n",
    "\n",
    "print(f\"Mean Absolute Error: {mae:.2f}\")\n",
    "print(f\"Mean Squared Error: {mse:.2f}\")\n",
    "print(f\"R2 Score: {r2:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Feature Importance Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Plot feature importance\n",
    "feature_importance = pd.DataFrame({\n",
    "    'feature': X.columns,\n",
    "    'importance': model.feature_importances_\n",
    "})\n",
    "feature_importance = feature_importance.sort_values('importance', ascending=False)\n",
    "\n",
    "plt.figure(figsize=(10, 6))\n",
    "sns.barplot(x='importance', y='feature', data=feature_importance.head(10))\n",
    "plt.title('Top 10 Most Important Features')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Save Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Save model and feature names\n",
    "model_data = {\n",
    "    'model': model,\n",
    "    'feature_names': X.columns.tolist(),\n",
    "    'feature_columns': feature_columns\n",
    "}\n",
    "\n",
    "joblib.dump(model_data, '../models/demand_model.pkl')\n",
    "print(\"Model saved to demand_model.pkl\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}